### Model Selection

In [4]:
import os

In [5]:
%pwd

'd:\\Silent_Night\\mlops\\AML_Classification\\research'

In [6]:
os.chdir("../")

In [7]:
%pwd

'd:\\Silent_Night\\mlops\\AML_Classification'

In [8]:
import pandas as pd
df=pd.read_csv("D:/data/classification_data/stress_data/stress_300.csv")
print(df.head())

   Humidity  Temperature  Step count  Stress Level
0     21.33        90.33         123             1
1     21.41        90.41          93             1
2     27.12        96.12         196             2
3     27.64        96.64         177             2
4     10.87        79.87          87             0


In [9]:
#Update entity(Specifiy the data type all the config and params yaml file variable)
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    #data_dir: Path
    base_model_path: Path
    params_n_estimators: list
    params_max_depth: list
    params_min_sample_split: list
    params_min_sample_leaf: list
    params_max_features: list
    params_class_weight: str

In [10]:
from AML_Classifier.constants.__init__ import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from AML_Classifier.utils.common import read_yaml, create_directories
from pathlib import Path

In [11]:
#Update the configration manager
from AML_Classifier.constants.__init__ import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from AML_Classifier.utils.common import read_yaml, create_directories
from pathlib import Path

class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        self.config= read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        #training_data = os.path.join(self.config.data_ingestion.root_dir,"stress_300.csv")

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            params_n_estimators = self.params.n_estimators,
            params_max_depth = self.params.max_depth,
            params_min_sample_split = self.params.min_samples_split,
            params_min_sample_leaf = self.params.min_samples_leaf,
            params_max_features = self.params.max_features,
            params_class_weight = self.params.class_weight
        )

        return prepare_base_model_config

In [12]:
#Update the compontents
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import metrics
import os
import pickle


In [13]:
class PrepareBaseModel:
    def __init__(self,config: PrepareBaseModelConfig):
        self.config = config

    def load_model_data(self):
        #Need to make this data ingestion dynamic
        data=df
        y=data[["Stress Level"]]
        x=data.drop(["Stress Level"],axis=1)

        # Split data into training and test sets
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

        # Define the parameter grid
        param_grid = {
            'n_estimators': self.config.params_n_estimators,
            #'max_depth': self.config.params_max_depth,
            #'min_samples_split': self.config.params_min_sample_split,
            #'min_samples_leaf': self.config.params_min_sample_leaf,
            #'max_features': self.config.params_max_features
            }
        
        # Initialize the classifier
        model = RandomForestClassifier()

        # Initialize GridSearchCV
        grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)

        # Fit the model
        grid_search.fit(X_train, y_train)

        # Print the best parameters
        print("Best Parameters:", grid_search.best_params_)

        # Evaluate the best model on the test set
        tuned_model=RandomForestClassifier(**grid_search.best_params_)
        tuned_model.fit(X_train,y_train)
        # Prediction Value
        y_pred=tuned_model.predict(X_test)
        #eval accuracy
        accuracy=metrics.accuracy_score(y_pred,y_test)
        print("Test Set Accuracy:", accuracy)
        

In [14]:
#pipeline
try:
    config = ConfigurationManager()
    prepare_base_model_config=config.get_prepare_base_model_config()
    prepare_base_model= PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.load_model_data()
except Exception as e:
    raise e

[2024-06-28 13:09:44,738: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-28 13:09:44,745: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-28 13:09:44,747: INFO: common: created directory at: artifacts]
[2024-06-28 13:09:44,749: INFO: common: created directory at: artifacts/prepare_base_model]
Fitting 3 folds for each of 3 candidates, totalling 9 fits


c:\Users\JOHN\anaconda3\envs\AML\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\JOHN\anaconda3\envs\AML\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Best Parameters: {'n_estimators': 100}
Test Set Accuracy: 1.0
